In [1]:
import pandas as pd
from IPython.display import display, HTML
import json

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# Function to merge columns with similar names
def merge_columns(df):
    # Group by the first word in the column name
    grouped = df.groupby(by=lambda x: x.split(" ")[0], axis=1)
    
    # Combine grouped columns
    for name, group in grouped:
        if len(group.columns) > 1:
            df[name] = group.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

    # Drop the original columns, keeping only the merged ones
    for name, group in grouped:
        if len(group.columns) > 1:
            df.drop(columns=group.columns[1:], inplace=True)
    return df

In [4]:
excel_path = 'CanIL Analysis of NT Conditionals by book220831.xlsx'

# Read all sheet names in the Excel file
xl = pd.ExcelFile(excel_path)
sheet_names = xl.sheet_names

# Skip the first sheet and last sheet, as it's just introductory material
sheet_names = sheet_names[1:-1]

# Read and concatenate all the sheets, making the column names lowercase
df_list = []
for sheet in sheet_names:
    df = pd.read_excel(excel_path, sheet_name=sheet)
    df.columns = map(str.lower, df.columns)  # Making column names lowercase
    df = merge_columns(df)  # Merge similar columns
    
    # Need to process the 'reference' column to add the sheet name + ' ' to the beginning of each value
    try:
        df['reference'] = sheet + ' ' + df['reference'].astype(str)
    except KeyError:
        print('Sheet name', sheet, 'does not have a "reference" column.')
    
    df_list.append(df)

In [5]:
# Concatenate all DataFrames in df_list
concatenated_df = pd.concat(df_list, ignore_index=True)

In [6]:
concatenated_df.head()

,reference,scope of conditional (esv unless noted),class,inv.,probability,time orientation,illocutionary force,english translations,notes,parallel passage(s),unnamed: 10,unnamed:,parallel passages,unnamed: 9,scope of conditional (esv unless otherwise indicated),scope of conditional (esv),scope of conditional (esv unless stated otherwise)
0,MAT 4:3,p: (if you are the Son of God)\nq: (command these stones to become bread),1,NaN,Factual,Present,Exhort,"ESV, NASB, NRSV, NIV, NLT: ""if""","P presents a fact that the temptor knew to be true. The devil certainly knows Jesus' identity. Hagner (1993) notes that this might as well be translated ""since"", as the devil is testing Jesus' obedience to the Father rather than questioning his identity. Q presents the subsequent command Jesus is being tempted to obey. The conditional as a whole amounts to a taunt or a challenge. Fong (2014: 30-31) discusses this as an example given by Young (1994) of a rhetorical conditional. That is, it is not being presented as a real condition, but the conditional is used for its logical connection in order to affect some other speech act, in this case ""manipulation"", according to Young. Fong, however, thinks it would be better classified as a ""challenge.""",Luke 4:3,NaN,,NaN,NaN,NaN,NaN,NaN
1,MAT 4:6,p: (if you are the Son of God)\nq: (throw yourself down),1,NaN,Factual,Present,Exhort,"ESV, NASB, NRSV, NIV, NLT: ""if""","As with 4:3, this conditional expresses no doubt about Jesus' identity, but is being used to manipulate, tempt, and challenge him.",Luke 4:9,NaN,,NaN,NaN,NaN,NaN,NaN
2,MAT 4:9,q:(All these I will give you)\np: (if you will fall down and worship me),3,x,Very Unlikely,Present,Promise / Exhort,"ESV, NASB, NRSV, NIV, NLT: ""if""","In 4:3 and 4:6 the exhortations (or temptations) are direct and occur in q; in this verse the exhortation (or temptation) is made indirectly in p. Though highly unlikely, and though it did not come to pass, this was still a (remote) possibility when the devil spoke it. It was a real option for Jesus, though he did not take it. Unlike the two previous temptations, this one is a third class, using εαν with the subjunctive because ( according to Hagner 1993) it involves an actual unmet condition.",Luke 4:7,NaN,,NaN,NaN,NaN,NaN,NaN
3,MAT 5:13a,p: (if salt has lost its taste)\nq: (how shall its saltiness be restored?),3,NaN,Unlikely,Gnomic,Assert,"ESV, NASB, NRSV, NIV, NLT: ""if""","This conditional must be read in its context of describing discipleship. The connection between salt and discipleship is that, as Marshall (1978) puts it, ""a false form of discipleship may look like salt, but the gradual process of leeching leaves only a zestless pile of waste."" Nolland (2005) claims that discussions of how salt of the day may have lost its flavour are pointless, as the point being made is that such a thing would be ""bizarre and unnatural."" It would be terrible for salt to lose saltiness because it is itself the thing that adds flavour to tasteless food.",Mark 9:50; Luke 14:34,NaN,,NaN,NaN,NaN,NaN,NaN
4,MAT 5:20,p: (unless your righteousness exceeds that of the scribes and Pharisees)\nq: (you will never enter the kingdom of heaven),3,NaN,Very Unlikely,Gnomic,Warn,"ESV, NASB, NRSV, NIV, NLT: ""unless""","Greek: εαν μη. Nolland, Hagner, and others agree that the righteousness Jesus discusses here cannot mean that he is actually commanding the disciples to live by the letter of the law as the Pharisees do (and to a greater extent, which is nigh impossible). \nParaphrase (to avoid multiple negatives in languages without 'unless' or 'except'): For I say to you that only if you are more righteous than the scribes and Pharisees, will you enter the kingdom of heaven.",NaN,NaN,,NaN,NaN,NaN,NaN,NaN


In [7]:
with open('output_Nov_6_ryder.jsonl', 'r', encoding='utf-8') as file:
    gloss_data = file.readlines()
    
gloss_data[:3]

['{"vref": "MAT 4:3", "data": {"protasis": [["n40004003007", "If", "Εἰ"], ["n40004003008", "Son", "υἱὸς"], ["n40004003009", "You are", "εἶ"], ["n40004003010", "-", "τοῦ"], ["n40004003011", "of God", "θεοῦ"]], "apodosis": [["n40004003012", "speak", "εἰπὲ"], ["n40004003013", "that", "ἵνα"], ["n40004003014", "the", "οἱ"], ["n40004003015", "stones", "λίθοι"], ["n40004003016", "these", "οὗτοι"], ["n40004003017", "loaves of bread", "ἄρτοι"], ["n40004003018", "might become", "γένωνται"]]}}\n',
 '{"vref": "MAT 4:6", "data": {"protasis": [["n40004006004", "If", "Εἰ"], ["n40004006005", "Son", "υἱὸς"], ["n40004006006", "You are", "εἶ"], ["n40004006007", "-", "τοῦ"], ["n40004006008", "of God", "θεοῦ"]], "apodosis": [["n40004006009", "throw", "βάλε"], ["n40004006010", "Yourself", "σεαυτὸν"], ["n40004006011", "down", "κάτω"]]}}\n',
 '{"vref": "MAT 4:9", "data": {"protasis": [["n40004009008", "if", "ἐὰν"], ["n40004009009", "falling down", "πεσὼν"], ["n40004009010", "You will worship", "προσκυνήσῃς"],

In [8]:
# Parse JSON strings into dictionaries
dict_list = [json.loads(json_str) for json_str in gloss_data]

# Flatten the dictionaries and create DataFrame
df = pd.json_normalize(dict_list)

# Rename the columns
df = df.rename(columns={'data.protasis': 'protasis', 'data.apodosis': 'apodosis'})

In [9]:
df.head()

,vref,protasis,apodosis,data.error,error
0,MAT 4:3,"[[n40004003007, If, Εἰ], [n40004003008, Son, υἱὸς], [n40004003009, You are, εἶ], [n40004003010, -, τοῦ], [n40004003011, of God, θεοῦ]]","[[n40004003012, speak, εἰπὲ], [n40004003013, that, ἵνα], [n40004003014, the, οἱ], [n40004003015, stones, λίθοι], [n40004003016, these, οὗτοι], [n40004003017, loaves of bread, ἄρτοι], [n40004003018, might become, γένωνται]]",NaN,NaN
1,MAT 4:6,"[[n40004006004, If, Εἰ], [n40004006005, Son, υἱὸς], [n40004006006, You are, εἶ], [n40004006007, -, τοῦ], [n40004006008, of God, θεοῦ]]","[[n40004006009, throw, βάλε], [n40004006010, Yourself, σεαυτὸν], [n40004006011, down, κάτω]]",NaN,NaN
2,MAT 4:9,"[[n40004009008, if, ἐὰν], [n40004009009, falling down, πεσὼν], [n40004009010, You will worship, προσκυνήσῃς], [n40004009011, me, μοι]]","[[n40004009004, These things, Ταῦτά], [n40004009005, to You, σοι], [n40004009006, all, πάντα], [n40004009007, will I give, δώσω]]",NaN,NaN
3,MAT 5:13,"[[n40005013007, if, ἐὰν], [n40005013010, the, τὸ], [n40005013011, becomes tasteless, μωρανθῇ]]","[[n40005013019, if, εἰ], [n40005013020, not, μὴ], [n40005013021, having been cast, βληθὲν], [n40005013022, out, ἔξω], [n40005013023, to be trampled upon, καταπατεῖσθαι]]",NaN,NaN
4,MAT 5:20,"[[n40005020004, that, ὅτι], [n40005020005, if, ἐὰν], [n40005020006, not, μὴ], [n40005020007, shall abound, περισσεύσῃ], [n40005020008, your, ὑμῶν], [n40005020009, -, ἡ], [n40005020010, righteousness, δικαιοσύνη], [n40005020011, above [that], πλεῖον], [n40005020012, of the, τῶν], [n40005020013, scribes, γραμματέων], [n40005020014, and, καὶ], [n40005020015, Pharisees, Φαρισαίων]]","[[n40005020016, no, οὐ], [n40005020017, not, μὴ], [n40005020018, shall you enter, εἰσέλθητε], [n40005020019, into, εἰς], [n40005020020, the, τὴν], [n40005020021, kingdom, βασιλείαν], [n40005020022, of the, τῶν], [n40005020023, heavens, οὐρανῶν]]",NaN,NaN


In [33]:
# Merge df and concatenated_df on 'reference'
merged_df = pd.merge(df, concatenated_df[['reference', 'scope of conditional (esv unless noted)']], left_on='vref', right_on='reference', how='left')

# Drop 'data.error', 'error', and 'reference' columns
merged_df = merged_df.drop(columns=['data.error', 'error', 'reference'])

In [34]:
merged_df = merged_df.rename(columns={'scope of conditional (esv unless noted)': 'p_q'})
merged_df.head()

,vref,protasis,apodosis,p_q
0,MAT 4:3,"[[n40004003007, If, Εἰ], [n40004003008, Son, υἱὸς], [n40004003009, You are, εἶ], [n40004003010, -, τοῦ], [n40004003011, of God, θεοῦ]]","[[n40004003012, speak, εἰπὲ], [n40004003013, that, ἵνα], [n40004003014, the, οἱ], [n40004003015, stones, λίθοι], [n40004003016, these, οὗτοι], [n40004003017, loaves of bread, ἄρτοι], [n40004003018, might become, γένωνται]]",p: (if you are the Son of God)\nq: (command these stones to become bread)
1,MAT 4:6,"[[n40004006004, If, Εἰ], [n40004006005, Son, υἱὸς], [n40004006006, You are, εἶ], [n40004006007, -, τοῦ], [n40004006008, of God, θεοῦ]]","[[n40004006009, throw, βάλε], [n40004006010, Yourself, σεαυτὸν], [n40004006011, down, κάτω]]",p: (if you are the Son of God)\nq: (throw yourself down)
2,MAT 4:9,"[[n40004009008, if, ἐὰν], [n40004009009, falling down, πεσὼν], [n40004009010, You will worship, προσκυνήσῃς], [n40004009011, me, μοι]]","[[n40004009004, These things, Ταῦτά], [n40004009005, to You, σοι], [n40004009006, all, πάντα], [n40004009007, will I give, δώσω]]",q:(All these I will give you)\np: (if you will fall down and worship me)
3,MAT 5:13,"[[n40005013007, if, ἐὰν], [n40005013010, the, τὸ], [n40005013011, becomes tasteless, μωρανθῇ]]","[[n40005013019, if, εἰ], [n40005013020, not, μὴ], [n40005013021, having been cast, βληθὲν], [n40005013022, out, ἔξω], [n40005013023, to be trampled upon, καταπατεῖσθαι]]",NaN
4,MAT 5:20,"[[n40005020004, that, ὅτι], [n40005020005, if, ἐὰν], [n40005020006, not, μὴ], [n40005020007, shall abound, περισσεύσῃ], [n40005020008, your, ὑμῶν], [n40005020009, -, ἡ], [n40005020010, righteousness, δικαιοσύνη], [n40005020011, above [that], πλεῖον], [n40005020012, of the, τῶν], [n40005020013, scribes, γραμματέων], [n40005020014, and, καὶ], [n40005020015, Pharisees, Φαρισαίων]]","[[n40005020016, no, οὐ], [n40005020017, not, μὴ], [n40005020018, shall you enter, εἰσέλθητε], [n40005020019, into, εἰς], [n40005020020, the, τὴν], [n40005020021, kingdom, βασιλείαν], [n40005020022, of the, τῶν], [n40005020023, heavens, οὐρανῶν]]",p: (unless your righteousness exceeds that of the scribes and Pharisees)\nq: (you will never enter the kingdom of heaven)


In [35]:
# Use regular expressions to extract 'p' and 'q' data
merged_df['p'] = merged_df['p_q'].str.extract(r'p:\s*\((.*?)\)', expand=False)
merged_df['q'] = merged_df['p_q'].str.extract(r'q:\s*\((.*?)\)', expand=False)

# Fill NaN values in both 'p' and 'q' columns with NaN
merged_df['p'].fillna("NaN", inplace=True)
merged_df['q'].fillna("NaN", inplace=True)

# Drop the original column
merged_df.drop('p_q', axis=1, inplace=True)

In [39]:
merged_df = merged_df.rename(columns={'protasis': 'protasis gloss'})
merged_df = merged_df.rename(columns={'apodosis': 'apodosis gloss'})
merged_df = merged_df.rename(columns={'p': 'protasis'})
merged_df = merged_df.rename(columns={'q': 'apodosis'})

In [38]:
merged_df = merged_df[['protasis', 'protasis gloss', 'apodosis', 'apodosis gloss']]
merged_df.head()

,protasis,protasis gloss,apodosis,apodosis gloss
0,if you are the Son of God,"[[n40004003007, If, Εἰ], [n40004003008, Son, υἱὸς], [n40004003009, You are, εἶ], [n40004003010, -, τοῦ], [n40004003011, of God, θεοῦ]]",command these stones to become bread,"[[n40004003012, speak, εἰπὲ], [n40004003013, that, ἵνα], [n40004003014, the, οἱ], [n40004003015, stones, λίθοι], [n40004003016, these, οὗτοι], [n40004003017, loaves of bread, ἄρτοι], [n40004003018, might become, γένωνται]]"
1,if you are the Son of God,"[[n40004006004, If, Εἰ], [n40004006005, Son, υἱὸς], [n40004006006, You are, εἶ], [n40004006007, -, τοῦ], [n40004006008, of God, θεοῦ]]",throw yourself down,"[[n40004006009, throw, βάλε], [n40004006010, Yourself, σεαυτὸν], [n40004006011, down, κάτω]]"
2,if you will fall down and worship me,"[[n40004009008, if, ἐὰν], [n40004009009, falling down, πεσὼν], [n40004009010, You will worship, προσκυνήσῃς], [n40004009011, me, μοι]]",All these I will give you,"[[n40004009004, These things, Ταῦτά], [n40004009005, to You, σοι], [n40004009006, all, πάντα], [n40004009007, will I give, δώσω]]"
3,NaN,"[[n40005013007, if, ἐὰν], [n40005013010, the, τὸ], [n40005013011, becomes tasteless, μωρανθῇ]]",NaN,"[[n40005013019, if, εἰ], [n40005013020, not, μὴ], [n40005013021, having been cast, βληθὲν], [n40005013022, out, ἔξω], [n40005013023, to be trampled upon, καταπατεῖσθαι]]"
4,unless your righteousness exceeds that of the scribes and Pharisees,"[[n40005020004, that, ὅτι], [n40005020005, if, ἐὰν], [n40005020006, not, μὴ], [n40005020007, shall abound, περισσεύσῃ], [n40005020008, your, ὑμῶν], [n40005020009, -, ἡ], [n40005020010, righteousness, δικαιοσύνη], [n40005020011, above [that], πλεῖον], [n40005020012, of the, τῶν], [n40005020013, scribes, γραμματέων], [n40005020014, and, καὶ], [n40005020015, Pharisees, Φαρισαίων]]",you will never enter the kingdom of heaven,"[[n40005020016, no, οὐ], [n40005020017, not, μὴ], [n40005020018, shall you enter, εἰσέλθητε], [n40005020019, into, εἰς], [n40005020020, the, τὴν], [n40005020021, kingdom, βασιλείαν], [n40005020022, of the, τῶν], [n40005020023, heavens, οὐρανῶν]]"


In [42]:
# Convert DataFrame to HTML
html_table = df.to_html(index=False)
with open('protasis_apodosis_table.html', 'a', encoding='utf-8') as file:
    file.write(html_table)